In [12]:
#install adapter
!pip install azure-storage-blob
!pip install boto3

In [13]:
import json
import boto3 # type: ignore
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient # type: ignore
from io import StringIO
import pandas as pd # type: ignore
import numpy as np # type: ignore
import requests # type: ignore

In [14]:
# Function

# Azure Functions
def azure_upload_blob(connect_str, container_name, blob_name, data):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded to Azure Blob: {blob_name}")

def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()

In [15]:
url ='https://data.wa.gov/resource/f6w7-q2d2.json'
df_raw=pd.read_json(url)
df_raw.head()

,vin_1_10,county,city,state,zip_code,model_year,make,model,ev_type,cafv_type,electric_range,base_msrp,legislative_district,dol_vehicle_id,geocoded_column,electric_utility,_2020_census_tract,:@computed_region_x4ys_rtnd,:@computed_region_fny7_vc3j,:@computed_region_8ddd_yn5v
0,5YJYGDEE1L,King,Seattle,WA,98122,2020,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,291,0,37,125701579,"{'type': 'Point', 'coordinates': [-122.30839, ...",CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),53033007800,3009,9,38
1,7SAYGDEE9P,Snohomish,Bothell,WA,98021,2023,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,1,244285107,"{'type': 'Point', 'coordinates': [-122.179458,...",PUGET SOUND ENERGY INC,53061051938,3213,1,1
2,5YJSA1E4XK,King,Seattle,WA,98109,2019,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,270,0,36,156773144,"{'type': 'Point', 'coordinates': [-122.34848, ...",CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),53033006800,3009,7,31
3,5YJSA1E27G,King,Issaquah,WA,98027,2016,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,210,0,5,165103011,"{'type': 'Point', 'coordinates': [-122.03646, ...",PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),53033032104,3009,8,6
4,5YJYGDEE5M,Kitsap,Suquamish,WA,98392,2021,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,23,205138552,"{'type': 'Point', 'coordinates': [-122.55717, ...",PUGET SOUND ENERGY INC,53035940100,848,6,29


In [16]:
print(df_raw.columns)

Index(['vin_1_10', 'county', 'city', 'state', 'zip_code', 'model_year', 'make',
       'model', 'ev_type', 'cafv_type', 'electric_range', 'base_msrp',
       'legislative_district', 'dol_vehicle_id', 'geocoded_column',
       'electric_utility', '_2020_census_tract', ':@computed_region_x4ys_rtnd',
       ':@computed_region_fny7_vc3j', ':@computed_region_8ddd_yn5v'],
      dtype='object')


In [17]:
# check null data

In [18]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   vin_1_10                     1000 non-null   object
 1   county                       1000 non-null   object
 2   city                         1000 non-null   object
 3   state                        1000 non-null   object
 4   zip_code                     1000 non-null   int64 
 5   model_year                   1000 non-null   int64 
 6   make                         1000 non-null   object
 7   model                        1000 non-null   object
 8   ev_type                      1000 non-null   object
 9   cafv_type                    1000 non-null   object
 10  electric_range               1000 non-null   int64 
 11  base_msrp                    1000 non-null   int64 
 12  legislative_district         1000 non-null   int64 
 13  dol_vehicle_id               1000 

In [19]:
#put into cloud storatge (create and upload)

In [20]:
# Specify the path to your JASON configuration file
config_file_path='config.json'

#Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

# Print the configuration
# Connection_STRING = config["connectionString"]


CONNECTION_STRING_AZURE_STORAGE = config["connectionString"]
CONTAINER_AZURE="electricalvehicle"
blob_name = "electricalvehicle_20240402.csv"
# Conbert DataFrame to CSV
output=StringIO()
df_raw.to_csv(output, index=False)
data = output.getvalue()
output.close()

# Create the BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Get a blob client using the container name and blob name
blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE, blob=blob_name)

# Upload the CSV data
blob_client.upload_blob(data, overwrite=True)

print(f"Uploaded {blob_name} to Azure Blob Storage in container {CONTAINER_AZURE}.")

Uploaded electricalvehicle_20240402.csv to Azure Blob Storage in container electricalvehicle.
